In [17]:
!pip install -r requirements.txt

In [18]:
# Bibliothèques standard
import os, requests, zipfile

# Data science et manipulation de données
import pandas as pd, numpy as np, geopandas as gpd

# Accès S3
import s3fs


# Fonctions et styles personnalisés importés de visualisation.py
from visualisation import (
    charte_graphique,
    charte_graphique2,
    charte_graphique3,
    dictionnaire_geo,
    tracer_evolution_taux,
    tracer_evolution_taux_relatif_lisse,
    boxplot_indicateur_par_saison,
    evolution_indicateur_animation,
    animer_evolution_densite,
    evolution_idf_animation,
    get_increase,
    tri_occurrence,
    plot_histogram,
    camembert,
    nb_lignes_traitant,
    filter_rows_with_keyword,
    count_crime_keywords,
    évolution_indicateur,
    évolution_indicateur,
)

In [19]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "anhlinh"

FILE_PATH_S3_DEP = f"{MY_BUCKET}/diffusion/df_indicateurs_dep.csv"
with fs.open(FILE_PATH_S3_DEP, "rb") as file_in_dep:
    df_indicateurs_dep = pd.read_csv(file_in_dep)


FILE_PATH_S3_LODA = f"{MY_BUCKET}/diffusion/df_loda.csv"
with fs.open(FILE_PATH_S3_LODA, "rb") as file_in_loda:
    df_loda = pd.read_csv(file_in_loda)

FILE_PATH_S3_NAT = f"{MY_BUCKET}/diffusion/df_indicateurs_nat.csv"
with fs.open(FILE_PATH_S3_NAT, "rb") as file_in_nat:
    df_indicateurs_nat = pd.read_csv(file_in_nat)


In [40]:
df_loda_reg = df_loda.drop([ "Unnamed: 0", "ID", "Date", "Nature", "Etat", "Origine", "Date Publication", "Mois"], axis = 1)
df_loda_reg.head()

,Titre,Année
0,Arrêté du 31 décembre 2008 relatif aux modalit...,2009
1,Arrêté du 31 décembre 2008 portant création d'...,2009
2,Décret n° 2008-1549 du 31 décembre 2008 portan...,2009
3,Arrêté du 31 décembre 2008 relatif aux modalit...,2009
4,Arrêté du 31 décembre 2008 relatif aux modalit...,2009


In [57]:
print(len(df_loda_reg))
df_loda_filtre = df_loda_reg[df_loda_reg["Titre"].str.contains(r"\bvols?\b", case=False, na=False)]
print(len(df_loda_filtre))

15541
231


In [58]:
df_loda_reg_filtre = df_loda_filtre.groupby("Année").size().reset_index(name="Nombre d'articles")
print(df_loda_reg_filtre)

    Année  Nombre d'articles
0    1997                  5
1    1998                  3
2    1999                  2
3    2000                  2
4    2001                 13
5    2002                  4
6    2004                  9
7    2005                 11
8    2006                  6
9    2008                 30
10   2009                 13
11   2010                  3
12   2011                 15
13   2012                 10
14   2013                 19
15   2014                  8
16   2015                  4
17   2016                  9
18   2017                 13
19   2018                  8
20   2019                  3
21   2020                  5
22   2021                  6
23   2022                 30


In [61]:
df_indicateurs_reg = df_indicateurs_dep.drop([ "Unnamed: 0", "Superficie (km2)", "Nombre" , "Département"], axis = 1)
df_indicateurs_reg = df_indicateurs_reg[df_indicateurs_reg["Indicateur"] == "Vols sans violence"]
df_indicateurs_reg.head()

,Année,Indicateur,Population,Taux (/10 000),Nom Département,Densité
10368,1996,Vols avec violence,499310,4.205804,Ain,86.655675
10369,1996,Vols avec violence,538017,6.003528,Aisne,73.080277
10370,1996,Vols avec violence,349376,5.695869,Allier,47.598910
10371,1996,Vols avec violence,137453,4.146872,Alpes-de-Haute-Provence,19.848809
10372,1996,Vols avec violence,118526,3.712266,Hautes-Alpes,21.359885


In [59]:
df_indicateurs_nat.head()
df_indicateurs_nat_reg = df_indicateurs_nat.loc[: ,["Année", "Taux de pauvreté (%)"]]
df_pauvrete_percent = df_indicateurs_nat_reg.drop_duplicates()
print(df_pauvrete_percent)

     Année  Taux de pauvreté (%)
0     1996                  19.3
12    1997                  18.8
24    1998                  18.4
36    1999                  17.9
48    2000                  18.0
60    2001                  17.2
72    2002                  16.6
84    2003                  18.4
96    2004                  18.0
108   2005                  18.8
120   2006                  18.0
132   2007                  18.2
144   2008                  18.5
156   2009                  19.0
168   2010                  18.9
180   2011                  19.2
192   2012                  20.5
204   2013                  20.0
216   2014                  20.2
228   2015                  19.7
240   2016                  19.7
252   2017                  19.5
264   2018                  19.6
276   2019                  19.7
288   2020                  18.6
300   2021                  20.2
312   2022                  19.3


In [63]:
df_pauvrete_loda_nbr= pd.merge(df_pauvrete_percent, df_loda_reg_filtre, on="Année", how="outer")
df_reg =pd.merge(df_pauvrete_loda_nbr, df_indicateurs_reg, on = "Année", how = "outer")
print(df_reg)

      Année  Taux de pauvreté (%)  Nombre d'articles          Indicateur  \
0      1996                  19.3                NaN  Vols avec violence   
1      1996                  19.3                NaN  Vols avec violence   
2      1996                  19.3                NaN  Vols avec violence   
3      1996                  19.3                NaN  Vols avec violence   
4      1996                  19.3                NaN  Vols avec violence   
...     ...                   ...                ...                 ...   
2587   2022                  19.3               30.0  Vols avec violence   
2588   2022                  19.3               30.0  Vols avec violence   
2589   2022                  19.3               30.0  Vols avec violence   
2590   2022                  19.3               30.0  Vols avec violence   
2591   2022                  19.3               30.0  Vols avec violence   

      Population  Taux (/10 000)          Nom Département      Densité  
0         4993

In [64]:
colonne_a_mettre_en_premier = "Nom Département"
colonnes = [colonne_a_mettre_en_premier] + [col for col in df_reg.columns if col != colonne_a_mettre_en_premier]
df_reg = df_reg[colonnes]
print(df_reg)

              Nom Département  Année  Taux de pauvreté (%)  Nombre d'articles  \
0                         Ain   1996                  19.3                NaN   
1                       Aisne   1996                  19.3                NaN   
2                      Allier   1996                  19.3                NaN   
3     Alpes-de-Haute-Provence   1996                  19.3                NaN   
4                Hautes-Alpes   1996                  19.3                NaN   
...                       ...    ...                   ...                ...   
2587                  Essonne   2022                  19.3               30.0   
2588           Hauts-de-Seine   2022                  19.3               30.0   
2589        Seine-Saint-Denis   2022                  19.3               30.0   
2590             Val-de-Marne   2022                  19.3               30.0   
2591               Val-d'Oise   2022                  19.3               30.0   

              Indicateur  P